In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from tqdm import tqdm 
from stellargraph import StellarDiGraph
from stellargraph.data import EdgeSplitter

ModuleNotFoundError: ignored

In [ ]:
edges = pd.read_csv("/content/drive/MyDrive/MLNS/edges.csv", sep = "\t")
edges.columns = ["source", "target"]

In [ ]:
node_info = pd.read_csv("/content/drive/MyDrive/MLNS/youtube_raw.csv", index_col = 0)
node_info.drop("id.1",inplace = True, axis = 1)

In [ ]:
node_info = node_info.iloc[:,0:8]

## Extract biggest connected component

In [ ]:
G = nx.Graph()
G.add_edges_from(edges.values.tolist())

#retrieve largest connected component
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G0 = G.subgraph(Gcc[0])

## Split in train and test

From the original graph G, extract a randomly sampled subset of test edges (true and false citation links) and the reduced graph G_test with the positive test edges removed.

The Test Graph is the reduced graph we obtain from removing the test set of links from the full graph.

In [ ]:
edge_splitter_test = EdgeSplitter(G0)
G_test, edge_ids_test, edge_labels_test =edge_splitter_test.train_test_split(
    p = 0.1, method = "global", keep_connected = True
)

** Sampled 85353 positive and 85353 negative edges. **


The reduced graph G_test, together with the test ground truth set of links (edge_ids_test, edge_labels_test), will be used for testing the model.

 From the reduced graph G_test, extract a randomly sampled subset of train edges (true and false citation links) and the reduced graph G_train with the positive train edges removed:

In [ ]:
edge_splitter_train = EdgeSplitter(G_test)
G_train, edge_ids_train, edge_labels_train =edge_splitter_train.train_test_split(
    p = 0.2, method = "global", keep_connected = True
)

** Sampled 153637 positive and 153637 negative edges. **


G_train, together with the train ground truth set of links (edge_ids_train, edge_labels_train), will be used for training the model.

G_train and G_test have the same set of nodes, only differing in their edge sets.


**Features for the train_set needs to  be computed on G_train, while feautures on the test_set needs to be computed on G_test**

In [ ]:
# if we want we can conver the list of edges with the corresponding label in a dataframe
# (not necessary, for example in the kaggle we created the features using lists)
test_edges = pd.DataFrame(edge_ids_test, columns = ["Source", "Target"])
test_edges["Edge"] = edge_labels_test
test_edges.head(5)

,Source,Target,Edge
0,Pp7Gdqvml5c,JbEyzt1vGGQ,1
1,s56wqvJomhI,aTRd97u4QsQ,1
2,horXN2RcIUk,nw8yokel7s4,1
3,uADZqpgu5nw,30cn9NcPkQc,1
4,sHsUWXR5MOM,J6cNegG6XyA,1


In [ ]:
train_edges = pd.DataFrame(edge_ids_train, columns = ["Source", "Target"])
train_edges["Edge"] = edge_labels_train
train_edges.head(5)

,Source,Target,Edge
0,OK-ecbojmGo,oUvRu3WMIa4,1
1,zUjdad3Yg2I,OyHXGQPBQm8,1
2,GdLTDChmE3U,vwjYiivrJdA,1
3,AYxXxqY3hoA,cV_zjocRnbo,1
4,zRQXdksI4qk,iaEXfJFNLx0,1


#Feature Extraction


## Node based features
- Channel name similarity
- Same uploader
- Age difference
- Same category
- Length difference
- Difference in views
- Difference in rate
- Difference in nb of ratings
- Difference in nb of comments

In [ ]:
#let's see if there are video uploaded by the same person
node_info.groupby("uploader").count().age.unique() # --> Yes, there are some uploader for which we have multiple videos

array([  1,   3,   2,   4,   8,   5,  13,   6,   7,  10,  11,  20,  18,
        34,  14,  30,  16,   9,  12,  19,  21,  17,  15,  23,  72,  24,
        47,  25,  22,  28,  45,  61,  46,  67,  26,  39,  71, 109,  32,
        33,  60,  27,  40,  29,  37,  38,  42,  65,  57,  74,  56,  31,
        53,  81,  79,  59,  41,  50,  55, 191,  44,  35,  77,  98,  36,
        43,  51])

In [ ]:
def node_features(edges):
  